<a href="https://colab.research.google.com/github/minhthy1016/Credit-Innovators/blob/minhthy1016-patch-1/get_all_inbox_message.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install \
 apache-airflow==1.10.10 \
 --constraint \
        https://raw.githubusercontent.com/apache/airflow/1.10.10/requirements/requirements-python3.7.txt

     |████████████████████████████████| 4.7MB 22.3MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 1.7MB 45.7MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 10.4MB 40.2MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 829kB 48.8MB/s 
     |████████████████████████████████| 450kB 43.9MB/s 
     |████████████████████████████████| 122kB 49.8MB/s 
     |████████████████████████████████| 327kB 50.4MB/s 
     |████████████████████████████████| 1.1MB 44.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparin

In [ ]:
import pandas as pd

df = pd.read_csv(r'/content/wp_postmeta.csv', error_bad_lines=False)
df = pd.DataFrame()
print(df)


b'Skipping line 55: expected 1 fields, saw 5\nSkipping line 102: expected 1 fields, saw 2\nSkipping line 181: expected 1 fields, saw 4\nSkipping line 185: expected 1 fields, saw 9\nSkipping line 189: expected 1 fields, saw 9\nSkipping line 191: expected 1 fields, saw 9\nSkipping line 469: expected 1 fields, saw 2\nSkipping line 477: expected 1 fields, saw 7\nSkipping line 483: expected 1 fields, saw 7\nSkipping line 1155: expected 1 fields, saw 29\nSkipping line 1173: expected 1 fields, saw 29\nSkipping line 1260: expected 1 fields, saw 10\nSkipping line 1435: expected 1 fields, saw 6\nSkipping line 1475: expected 1 fields, saw 11\nSkipping line 1619: expected 1 fields, saw 5\nSkipping line 1660: expected 1 fields, saw 5\nSkipping line 1686: expected 1 fields, saw 7\nSkipping line 1698: expected 1 fields, saw 4\nSkipping line 1719: expected 1 fields, saw 5\nSkipping line 1735: expected 1 fields, saw 2\nSkipping line 1848: expected 1 fields, saw 6\nSkipping line 1875: expected 1 fields,

Empty DataFrame
Columns: []
Index: []


In [ ]:
import datetime
all_token_df = pd.read_excel(r'/content/token_list.xlsx')

names = ["MarryBaby", "Hello Bacsi", "Hello Sehat", "Hello คุณหมอ", "Hello स्वास्थ्य", "Hello माँ और शिशु",
         "Hello Doktor", "Hello醫師", "Hello Mommy", "Hello គ្រូពេទ្យ", "Hello အမ်ိဳးသားက်န္းမာေရး", "Hello ဆရာ၀န္",
         "Hello မိခင္ႏွင့္ကေလး", "Hello Doctor"]

ytd = datetime.datetime.today() - datetime.timedelta(1)

ytd_begin = datetime.datetime(ytd.year, ytd.month, ytd.day)
ytd_end = datetime.datetime(ytd.year, ytd.month, ytd.day) + datetime.timedelta(1)

all_token_df = all_token_df[all_token_df['name'].isin(names)]
print(ytd)
print(ytd_begin,ytd_end)
all_token_df = pd.DataFrame()
print(all_token_df)


2021-04-13 07:57:54.899223
2021-04-13 00:00:00 2021-04-14 00:00:00
Empty DataFrame
Columns: []
Index: []


In [ ]:
import logging

import os
import time
import requests
import json
import pandas as pd
import pandas_gbq
import google.auth
import google.auth.transport.requests
from google.cloud import bigquery
from datetime import date, timedelta

import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
from requests.adapters import HTTPAdapter
#from requests.packages.urllib3.util.retry import Retry
from urllib3.util import Retry
from pprint import pprint
import pandas as pd
import http
import datetime
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest


all_token_df = pd.read_excel(r'/content/token_list.xlsx')

names = ["MarryBaby", "Hello Bacsi", "Hello Sehat", "Hello คุณหมอ", "Hello स्वास्थ्य", "Hello माँ और शिशु",
         "Hello Doktor", "Hello醫師", "Hello Mommy", "Hello គ្រូពេទ្យ", "Hello အမ်ိဳးသားက်န္းမာေရး", "Hello ဆရာ၀န္",
         "Hello မိခင္ႏွင့္ကေလး", "Hello Doctor"]

ytd = datetime.datetime.today() - datetime.timedelta(1)

ytd_begin = datetime.datetime(ytd.year, ytd.month, ytd.day)
ytd_end = datetime.datetime(ytd.year, ytd.month, ytd.day) + datetime.timedelta(1)

all_token_df = all_token_df[all_token_df['name'].isin(names)]



class inbox_scrapper:
    def __init__(self, page_name, id, token, table_name, since, to):
        self.page_name = page_name
        self.id = id
        self.token = token
        self.ids = []
        self.table_name = table_name
        self.session = requests.Session()
        if type(since) is str:
            self.since = datetime.datetime.strptime(since, '%Y-%m-%d')
        else:
            self.since = since
        if type(to) is str:
            self.to = datetime.datetime.strptime(to, '%Y-%m-%d')
        else:
            self.to = to

        retries = Retry(total=15,
                        backoff_factor=0.1,
                        status_forcelist=[500, 502, 503, 504])

        self.session.mount('http://', HTTPAdapter(max_retries=retries))
        self.session.mount('https://', HTTPAdapter(max_retries=retries))
        self.credentials = service_account.Credentials.from_service_account_file(r'/content/hhg-cdm-bcdeee60c0e1.json')
        self.schema = [{'name': 'time', 'type': 'DATETIME'}]

    def get_convo(self):
        params = (
            ('fields', 'conversations'),
            ('access_token', self.token),
            ('limit', 40000))

        response = self.session.get('https://graph.facebook.com/v9.0/{}'.format(self.id), params=params)
        # print(response.json())
        data = response.json().get('conversations').get('data')
        next = response.json().get('conversations').get('paging').get('next')
        print(next)
        while data and next:
            self.ids.extend([convo.get('id') for convo in data if (datetime.datetime.strptime(convo.get('updated_time'),
                                                                                              '%Y-%m-%dT%H:%M:%S+0000') >= self.since and datetime.datetime.strptime(
                convo.get('updated_time'), '%Y-%m-%dT%H:%M:%S+0000') < self.to)])

            if datetime.datetime.strptime(data[0].get('updated_time'), '%Y-%m-%dT%H:%M:%S+0000') < self.since:
                data = None
            else:
                response = self.session.get(next)
                print(response.json())
                data = response.json().get('data')
                print(data)
                if data:
                    next = response.json().get('paging').get('next')
                else:
                    next = None

        print('Number of convo: ', len(self.ids))

    def get_messages(self, id, token):
        all_data = []
        response = self.session.get(
            'https://graph.facebook.com/v9.0/{0}/messages?fields=id,from,created_time,message&limit=40000&access_token={1}'.format(
                id, token))
        data = response.json().get('data')

        while data:
            all_data.extend(data)
            next = response.json().get('paging').get('cursors').get('after')
            response = self.session.get(
                'https://graph.facebook.com/v9.0/{0}/messages?fields=id,from,created_time,message&limit=40000&access_token={1}&after={2}'.format(
                    id, token, next))
            data = response.json().get('data')
        return all_data


    def get_all_messages(self):
        self.all_messages = pd.DataFrame()
        for id in self.ids:
            messages = self.get_messages(id, self.token)
            df = pd.DataFrame(messages)
            print(df)
            df['page'] = self.page_name
            df['user_name'] = df['from'].apply(lambda x: x.get('name'))
            df['user_id'] = df['from'].apply(lambda x: x.get('id'))
            df['conversation_id'] = id
            df['time'] = df[
                'created_time']  # .apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+0000').replace(tzinfo=datetime.timezone.utc))
            df = df.drop(columns=['from', 'created_time'])
            self.all_messages = self.all_messages.append(df)

        records=self.all_messages.to_dict('records')
        if len(records)>0:
            self.streaming_to_bq(records)

        # rec=self.all_messages.to_dict('records')[0]
        # print(rec['time'].strftime('%Y-%m-%dT%H:%M:%S+0000'))
        #try:
           # self.streaming_to_bq(self.all_messages.to_dict('records'))
        #except:
            #pass


    def __repr__(self):
        return f'inbox_scrapper(id={self.id}, token={self.token}, table_name=\'hhg_project_social_media.{self.table_name}\')'

    def streaming_to_bq(self, rows_to_insert, table_id='hhg-ads.hhg_project_social_media.all_inbox_di'):
        # delete table
        client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')
        # streaming data
        errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
        if errors == []:
            print("New rows have been added for: ", table_id)
        else:
            print("Encountered errors while inserting rows: {}".format(errors))


def main():
  for index, row in all_token_df.iterrows():
      print(row['name'], end='\r')
      page_name=row['name']
      page_id=row['id']
      page_token=row['access_token']
      scrapper=inbox_scrapper(page_name,page_id,page_token, 'all_inbox_di', ytd_begin, ytd_end)
      scrapper.get_convo()
      scrapper.get_all_messages()

print(df)




Empty DataFrame
Columns: []
Index: []


In [ ]:
import logging

import os
import time
import requests
import json
import pandas as pd
import pandas_gbq
import google.auth
import google.auth.transport.requests
from google.cloud import bigquery
from datetime import date, timedelta

import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
from requests.adapters import HTTPAdapter
#from requests.packages.urllib3.util.retry import Retry
from urllib3.util import Retry
from pprint import pprint
import pandas as pd
import http
import datetime
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest


all_token_df = pd.read_excel(r'/content/token_list.xlsx')

names = ["MarryBaby", "Hello Bacsi", "Hello Sehat", "Hello คุณหมอ", "Hello स्वास्थ्य", "Hello माँ और शिशु",
         "Hello Doktor", "Hello醫師", "Hello Mommy", "Hello គ្រូពេទ្យ", "Hello အမ်ိဳးသားက်န္းမာေရး", "Hello ဆရာ၀န္",
         "Hello မိခင္ႏွင့္ကေလး", "Hello Doctor"]

ytd = datetime.datetime.today() - datetime.timedelta(1)

ytd_begin = datetime.datetime(ytd.year, ytd.month, ytd.day)
ytd_end = datetime.datetime(ytd.year, ytd.month, ytd.day) + datetime.timedelta(1)

all_token_df = all_token_df[all_token_df['name'].isin(names)]



class inbox_scrapper:
    def __init__(self, page_name, id, token, table_name, since, to):
        self.page_name = page_name
        self.id = id
        self.token = token
        self.ids = []
        self.table_name = table_name
        self.session = requests.Session()
        if type(since) is str:
            self.since = datetime.datetime.strptime(since, '%Y-%m-%d')
        else:
            self.since = since
        if type(to) is str:
            self.to = datetime.datetime.strptime(to, '%Y-%m-%d')
        else:
            self.to = to

        retries = Retry(total=15,
                        backoff_factor=0.1,
                        status_forcelist=[500, 502, 503, 504])

        self.session.mount('http://', HTTPAdapter(max_retries=retries))
        self.session.mount('https://', HTTPAdapter(max_retries=retries))
        self.credentials = service_account.Credentials.from_service_account_file(r'/content/hhg-cdm-bcdeee60c0e1.json')
        self.schema = [{'name': 'time', 'type': 'DATETIME'}]

    def get_convo(self):
        params = (
            ('fields', 'conversations'),
            ('access_token', self.token),
            ('limit', 40000))

        response = self.session.get('https://graph.facebook.com/v9.0/{}'.format(self.id), params=params)
        # print(response.json())
        data = response.json().get('conversations').get('data')
        next = response.json().get('conversations').get('paging').get('next')
        print(next)
        while data and next:
            self.ids.extend([convo.get('id') for convo in data if (datetime.datetime.strptime(convo.get('updated_time'),
                                                                                              '%Y-%m-%dT%H:%M:%S+0000') >= self.since and datetime.datetime.strptime(
                convo.get('updated_time'), '%Y-%m-%dT%H:%M:%S+0000') < self.to)])

            if datetime.datetime.strptime(data[0].get('updated_time'), '%Y-%m-%dT%H:%M:%S+0000') < self.since:
                data = None
            else:
                response = self.session.get(next)
                print(response.json())
                data = response.json().get('data')
                print(data)
                if data:
                    next = response.json().get('paging').get('next')
                else:
                    next = None

        print('Number of convo: ', len(self.ids))

    def get_messages(self, id, token):
        all_data = []
        response = self.session.get(
            'https://graph.facebook.com/v9.0/{0}/messages?fields=id,from,created_time,message&limit=40000&access_token={1}'.format(
                id, token))
        data = response.json().get('data')

        while data:
            all_data.extend(data)
            next = response.json().get('paging').get('cursors').get('after')
            response = self.session.get(
                'https://graph.facebook.com/v9.0/{0}/messages?fields=id,from,created_time,message&limit=40000&access_token={1}&after={2}'.format(
                    id, token, next))
            data = response.json().get('data')
        return all_data


    def get_all_messages(self):
        self.all_messages = pd.DataFrame()
        for id in self.ids:
            messages = self.get_messages(id, self.token)
            df = pd.DataFrame(messages)
            print(df)
            df['page'] = self.page_name
            df['user_name'] = df['from'].apply(lambda x: x.get('name'))
            df['user_id'] = df['from'].apply(lambda x: x.get('id'))
            df['conversation_id'] = id
            df['time'] = df[
                'created_time']  # .apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+0000').replace(tzinfo=datetime.timezone.utc))
            df = df.drop(columns=['from', 'created_time'])
            self.all_messages = self.all_messages.append(df)

        records=self.all_messages.to_dict('records')
        if len(records)>0:
            self.streaming_to_bq(records)

        # rec=self.all_messages.to_dict('records')[0]
        # print(rec['time'].strftime('%Y-%m-%dT%H:%M:%S+0000'))
        #try:
           # self.streaming_to_bq(self.all_messages.to_dict('records'))
        #except:
            #pass

    def __repr__(self):
        return f'inbox_scrapper(id={self.id}, token={self.token}, table_name=\'hhg_project_social_media.{self.table_name}\')'

    def streaming_to_bq(self, rows_to_insert, table_id='hhg-ads.hhg_project_social_media.all_inbox_di'):
        # delete table
        client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')
        client.delete_table(table_id, not_found_ok=True)  # Make an API request.
        print("Deleted table '{}'.".format(table_id))
        


    def main():
      for index, row in all_token_df.iterrows():
          print(row['name'], end='\r')
          page_name=row['name']
          page_id=row['id']
          page_token=row['access_token']
          scrapper=inbox_scrapper(page_name,page_id,page_token, 'all_inbox_di', ytd_begin, ytd_end)
          scrapper.get_convo()
          scrapper.get_all_messages()



###############################################

    def main():
        credentials = service_account.Credentials.from_service_account_file(
            r'/content/hhg-cdm-bcdeee60c0e1.json')
        for index, row in all_token_df.iterrows():
          print(row['name'], end='\r')
          page_name=row['name']
          page_id=row['id']
          page_token=row['access_token']
          scrapper=inbox_scrapper(page_name,page_id,page_token, 'all_inbox_di', ytd_begin, ytd_end)
          scrapper.get_convo()
          all_msg = scrapper.get_all_messages()
          pprint(all_msg)

          msg_schema=[
          bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
          bigquery.SchemaField("message", "STRING", mode="NULLABLE"),
          bigquery.SchemaField("page", "STRING", mode="NULLABLE"),
          bigquery.SchemaField("user_name", "STRING", mode="NULLABLE"),
          bigquery.SchemaField("user_id", "STRING", mode="NULLABLE"),
          bigquery.SchemaField("conversation_id", "STRING", mode="NULLABLE"),
          bigquery.SchemaField("time", "DATETIME", mode="NULLABLE"),
          ]
        #streaming_to_bq('hhg-ods.test.mailchimp_content', all_content, content_schema)
        streaming_to_bq('hhg-ads.hhg_project_social_media.all_inbox_di', all_msg, msg_schema)
        #create table
        table = bigquery.Table(self, rows_to_insert, table_id, schema=msg_schema)
        table = client.create_table(table)  # Make an API request.
        print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
            # streaming data
        errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
        if errors == []:
            print("New rows have been added for: ", table_id)
        else:
            print("Encountered errors while inserting rows: {}".format(errors))

    main()




https://graph.facebook.com/v9.0/107768247481687/conversations?access_token=EAAB7YhYjVW0BAIlrlg7Faj9TxE9RXZBFcOObfp0ZBS5CDY31JDfDMniRZCeM27W2WxuhybgNltminZA3zTBePbQJO9Oy413KOtYVpYpjWav1X7natCVK3eEwLnWsXMxYeoC7xgwpvYrzod5KSW2IZC4eQFgAOFZBa1JMtUeNzZAE8ZCHC2LSIeDP&limit=25&after=QVFIUjZA6SDJpUWtaV3c2OXJjUTBDU2VfUTF6ZAGJpRGE5dzZAzTS1SR3RGWGI0Qm9kZAlFnSEZAmNVk2eERjYWp1RTBJTWxqM2p6Tk50UnVEZAXE2Qm94MjY1SHR6cE4xa0FzSklSd3FUa2dwbFZAZAOG9oR3Ftc0RKcExRS2RlVmN6bGRNR3FM
{'data': [{'id': 't_2152907514845024', 'link': '/107768247481687/inbox/255853319339845/', 'updated_time': '2021-02-23T07:17:51+0000'}, {'id': 't_3765926310163513', 'link': '/107768247481687/inbox/255087866083057/', 'updated_time': '2021-02-21T21:07:56+0000'}, {'id': 't_1883974671782816', 'link': '/107768247481687/inbox/254293152829195/', 'updated_time': '2021-02-20T13:20:24+0000'}, {'id': 't_228776602294241', 'link': '/107768247481687/inbox/253710536220790/', 'updated_time': '2021-02-19T14:01:55+0000'}, {'id': 't_461804351865194', 'l

TypeError: ignored